Code to prepare and plot the altimetry input data files, both for Diva2D and [divand](https://www.github.com/gher-ulg/divand.jl/).

In [1]:
import os
import re
import datetime
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib import rcParams
import netCDF4
import logging
import divaaltimetry
from calendar import monthrange

## Configuration

Logging

In [2]:
logger = logging.getLogger("diva-altimetry")
logger.setLevel(logging.DEBUG)

Plotting (should be ideally in `matplotlibrc` but seems not to work with virtualenv).

In [3]:
rcParams.update({'font.size': 16, 'figure.dpi': 300, 'savefig.bbox': 'tight'})

# Input parameters

## Files and directories

In [4]:
databasedir = "/home/ctroupin/Data/AVISO/regional-mediterranean/delayed-time/along-track/filtered/adt"
outputdir = "../data/"
figdir = "../figures/"
missionlist = ["al", "c2", "h2", "j2"]

## Domain and period of interest
* coordinates: tuple with lon_min, lon_max, lat_min, lat_max 
* timescale: temporal scale in days; used to compute weights for Diva2D

In [16]:
coordinates = (-6.75, 40.001, 30, 48.)
yearlist, monthlist, daylist, interval = [2014], range(1, 13), range(1, 32), 10
timescale = 5.        

## Map projection
Define the projection from the coordinates; used for the plots.

In [14]:
m = Basemap(projection='merc',
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

## Generate list of files for the selected period

Based on the filenames, we can easily identify the files corresponding to the time of interest:
* datemid is the central time of the considered period
* datestart is the start date
* dateend is the last day.

# Main loop
Loop on years, months and days; for each day, create a list of the files that contains the dates of interest (day +/- the selected interval).

In [24]:
for year in set(yearlist):
    
    for month in set(monthlist):
        daystart = 1 
        dayend = monthrange(year, month)[1]
        if month == monthlist[0]:
            daystart = daylist[0]
        if month == monthlist[-1]:
            dayend = daylist[-1]
        
        for day in range(daystart, dayend + 1):
            logger.debug("year: {0}, month: {1}, day: {2}".format(year, month, day))
            filelist = divaaltimetry.make_filelist(databasedir, missionlist, 
                                                   year, month, day, interval)

            datemid = datetime.datetime(year, month, day).toordinal() - \
            datetime.datetime(1950, 1, 1).toordinal()
            logging.info("Central time (in days since January 1st, 1950): {0}".
                         format(datemid))

            fignamesuffix, figtitledate = divaaltimetry.prepare_datestrings(year, 
                                                                            month, 
                                                                            day, interval)
            logging.info(fignamesuffix)

            ## File and figure names
            #Generate file and figure names using the suffixes.
            outputfilename = "ADT_{0}.dat".format(fignamesuffix)
            divafilename =  "data_{0}.dat".format(fignamesuffix)
            figname = "ADT_{0}".format(fignamesuffix)
            logging.info("Text file name: {0}".format(outputfilename))
            logging.info("Figure name: {0}".format(figname))

            ## Make the plot and write data to text files
            """
            divaaltimetry.plot_data_tracklist(filelist, 
                                              os.path.join(figdir, figname), 
                                              figtitledate, 
                                              m)
            """
            divaaltimetry.write_tracks2file(filelist, 
                                            os.path.join(outputdir, outputfilename), 
                                            os.path.join(outputdir, divafilename), 
                                            timescale, datemid)

DEBUG:diva-altimetry:year: 2014, month: 1, day: 1
DEBUG:diva-altimetry:year: 2014, month: 1, day: 2
DEBUG:diva-altimetry:year: 2014, month: 1, day: 3
DEBUG:diva-altimetry:year: 2014, month: 1, day: 4
DEBUG:diva-altimetry:year: 2014, month: 1, day: 5
DEBUG:diva-altimetry:year: 2014, month: 1, day: 6
DEBUG:diva-altimetry:year: 2014, month: 1, day: 7
DEBUG:diva-altimetry:year: 2014, month: 1, day: 8
DEBUG:diva-altimetry:year: 2014, month: 1, day: 9
DEBUG:diva-altimetry:year: 2014, month: 1, day: 10
DEBUG:diva-altimetry:year: 2014, month: 1, day: 11
DEBUG:diva-altimetry:year: 2014, month: 1, day: 12
DEBUG:diva-altimetry:year: 2014, month: 1, day: 13
DEBUG:diva-altimetry:year: 2014, month: 1, day: 14
DEBUG:diva-altimetry:year: 2014, month: 1, day: 15
DEBUG:diva-altimetry:year: 2014, month: 1, day: 16
DEBUG:diva-altimetry:year: 2014, month: 1, day: 17
DEBUG:diva-altimetry:year: 2014, month: 1, day: 18
DEBUG:diva-altimetry:year: 2014, month: 1, day: 19
DEBUG:diva-altimetry:year: 2014, month: 